# 目的
今後解析を進めるにあたって、csvでやり取りする（pandas）か、RDB（sqlite）でやり取りするかという選択肢が出た。   
csvの場合、エクセルやメモ帳でデータの確認ができるというメリットがある。   
一方RDBの場合、データの確認はSQL queryを通す他ないが検索や処理が一文で済んでしまうというメリットがある。   
機械学習などの分野ではRDBよりcsvが好まれるが、RDBは科学計算や一般的な企業でデータ管理システムとして広く利用されている（Oracleに代表される）。   

今回はPythonを用いたRDBの練習もかねて、SQLite3を利用したFX用データベースおよびデータ処理システムの構築を目指す。   
さしあたって最初は、csvで保存してきたデータをひとまとめにする。

## サンプルコード

In [5]:
# -*- coding: utf-8 -*-

import sqlite3

dbname = 'database.db'

conn = sqlite3.connect(dbname)
c = conn.cursor()

# executeメソッドでSQL文を実行する
create_table = '''create table users (id int, name varchar(64),
                  age int, gender varchar(32))'''
c.execute(create_table)
print(c.fetchall())

# SQL文に値をセットする場合は，Pythonのformatメソッドなどは使わずに，
# セットしたい場所に?を記述し，executeメソッドの第2引数に?に当てはめる値を
# タプルで渡す．
sql = 'insert into users (id, name, age, gender) values (?,?,?,?)'
user = (1, 'Taro', 20, 'male')
c.execute(sql, user)

# 一度に複数のSQL文を実行したいときは，タプルのリストを作成した上で
# executemanyメソッドを実行する
insert_sql = 'insert into users (id, name, age, gender) values (?,?,?,?)'
users = [
    (2, 'Shota', 54, 'male'),
    (3, 'Nana', 40, 'female'),
    (4, 'Tooru', 78, 'male'),
    (5, 'Saki', 31, 'female')
]
c.executemany(insert_sql, users)
conn.commit()

select_sql = 'select * from users'
for row in c.execute(select_sql):
    print(row)

conn.close()

[]
(1, 'Taro', 20, 'male')
(2, 'Shota', 54, 'male')
(3, 'Nana', 40, 'female')
(4, 'Tooru', 78, 'male')
(5, 'Saki', 31, 'female')


## クラスの構築

In [98]:
import numpy as np
import pandas as pd
import sqlite3
import os
import time
import datetime
from FX.FX.core.misc import *

In [165]:
class SQLBaseforFX(object):
    """
    The base class of SQLDB for FX.
    This class mainly has methods processing the "main" table,
    which has the structure (datetime, dateval, ask, bid).
    """
    def __init__(self, currencyPair="usdjpy", recreate=False):
        """
        Initialization
        If a database named `sef._dbname` doesn't exist, then it will be created first.
        If `recreate` is true, then the database will be deleted and recreated.
        """
        self._currencyPair = currencyPair
        now = datetime.datetime.now()
        self._dbname = "../data/{2}_{0}{1:02d}.db".format(now.year, now.month, self._currencyPair)
        if not os.path.exists(self._dbname):
            try:
                conn = sqlite3.connect(self._dbname)
                conn.close()
            except Exception as e:
                print(e)
        elif recreate is True:
            os.remove(self._dbname)
            try:
                conn = sqlite3.connect(self._dbname)
                conn.close()
            except Exception as e:
                print(e)
        
        # Make the main table.
        self._tblmain = "main"
        self._mainstruct = "(datetime varchar(255), dateval int, ask real, bid real)"
        self.maketable(self._tblmain, self._mainstruct)
        
        self._dtformat = "%Y-%m-%d %H:%M:%S"
    
    def maketable(self, tblname, structure):
        """
        Make a table named `tblname` with the structure `structure`.
        """
        conn = sqlite3.connect(self._dbname)
        c = conn.cursor()
        try:
            c.execute("select name from sqlite_master where type='table'")
            table_names = [row[0] for row in c.fetchall()]
            if tblname not in table_names:
                create_table = '''create table {0} {1}'''.format(tblname, structure)
                c.execute(create_table)
                conn.commit()
        except Exception as e:
            print("Error:", e)
        conn.close()
    
    def addFXRecordsFromFolder(self, fldrpath):
        """
        Add (, or insert) the records in the all file `fpath` which has the following datasets:
            datetime (string), ask (float), bid (float).
        The structure of the main table is:
            (datetime varchar(255), dateval int, ask real, bid real)
        """
        filelist = os.listdir(fldrpath)
        print("Start insertion of the data in folder: {}".format(fldrpath))
        st = time.time()
        for fname in filelist:
            self.addFXRecordsFromFile(os.path.join(fldrpath, fname))
        print("Finished. Elapsed time: {0:.2f} sec.".format(time.time()-st))
        
    def addFXRecordsFromFile(self, fpath):
        """
        Add (, or insert) the records in the file `fpath` which has the following datasets:
            datetime (string), ask (float), bid (float).
        The structure of the main table is:
            (datetime varchar(255), dateval int, ask real, bid real)
        """
        _ = pd.read_csv(fpath, index_col=0)
        data = _.as_matrix()
        timestamps = list(_.index)
        dates = [datetime2time2(datetime.datetime.strptime(s, self._dtformat))
                 for s in timestamps]
        dataset = [(timestamps[ii], dates[ii], data[ii, 0], data[ii,1]) for ii in range(len(dates))]
        
        insert_sql = '''insert into {} (datetime, dateval, ask, bid) values (?,?,?,?)'''.format(self._tblmain)
        self.executemany(insert_sql, dataset)
               
    def addFXRecord(self, dt, ask, bid):
        """
        Add (, or insert) a record (dt, dt->dateval, ask, bid).
        The structure of the main table is:
            (datetime varchar(255), dateval int, ask real, bid real)
        """
        datetimestr = dt.strftime(self._dtformat)
        date = datetime2time2(dt)
        dataset = (datetimestr, date, ask, bid)
        insert_sql = '''insert into {} (datetime, dateval, ask, bid) values (?,?,?,?)'''.format(self._tblmain)
        res = self.execute(insert_sql, dataset)
        
    def toArray(self, tblname=None):
        """
        Convert the table `tblname` to an array.
        The default table to convert is `self._tblmain`.
        If the target table is 'main', the first column 'datatime' is ignored.
        """
        if tblname is None:
            tblname = self._tblmain
        if tblname == "main":
            select_sql = '''select dateval, ask, bid from {}'''.format(tblname)
        else:
            select_sql = '''select * from {}'''.format(tblname)
        res = self.execute(select_sql)
        return np.array(res)
    
    def toDataFrame(self, tblname=None):
        """
        Convert the table `tblname` to a DataFrame object.
        The default table to convert is `self._tblmain`.
        """
        if tblname is None:
            tblname = self._tblmain
        if tblname == "main":
            select_sql = '''select datetime from {}'''.format(tblname)
            _ = self.execute(select_sql)
            nowtimes = [n[0] for n in _]
            select_sql = '''select dateval, ask, bid from {}'''.format(tblname)
            _ = self.execute(select_sql)
            data = np.array(_)
            df = pd.DataFrame(data, columns=["dateval", "ask", "bid"])
            df.index = nowtimes
        else: # 修正が必要：テーブルによらず、datetimeが入る場合に、それをインデックスに指定する
            res = self.execute("PRAGMA table_info({})".format(tblname))
            colnames = [row[1] for row in res]
            select_sql = '''select * from {}'''.format(tblname)
            _ = self.execute(select_sql)
            data = np.array(_)
            df = pd.DataFrame(data, columns=colnames)
            
        return df
    
    def toCSV(self, savepath, tblname=None):
        """
        Save the table `tblname` to the file `savepath`.
        """
        df = self.toDataFrame(tblname)
        df.to_csv(savepath)
    
    def execute(self, sql_txt, value=None):
        """
        Execute a SQL query `sql_txt`.
        """
        res = None
        conn = sqlite3.connect(self._dbname)
        c = conn.cursor()
        try:
            if value is None:
                c.execute(sql_txt)
            else:
                print(value)
                c.execute(sql_txt, value)
            res = c.fetchall()
            conn.commit()
        except Exception as e:
            print("Error:", e)
        conn.close()
        return res
    
    def executemany(self, sql_txt, value):
        """
        Execute a SQL query `sql_txt` with many datasets `value`.
        """
        res = None
        conn = sqlite3.connect(self._dbname)
        c = conn.cursor()
        try:
            c.executemany(sql_txt, value)
            res = c.fetchall()
            conn.commit()
        except Exception as e:
            print("Error:", e)
        conn.close()
        return res

## テストコーナー

### 初期化

In [166]:
sql = SQLforFX(recreate=False)

### レコードの挿入

In [ ]:
sql.addFXRecord(datetime.datetime.now(), 111, 110.93)

In [ ]:
sql.addFXRecordsFromFile("../data/201704/fxdata_20170401_005213.csv")

In [106]:
sql.addFXRecordsFromFolder("../data/201704/")

Start insertion of the data in folder: ../data/201704/
Finished. Elapsed time: 63.96 sec.


### SQL文の実行

In [167]:
res = sql.execute("select ask from main")

In [168]:
res = sql.execute("PRAGMA table_info(main)")
colnames = [row[1] for row in res]
print(colnames)

['datetime', 'dateval', 'ask', 'bid']


### 配列などへの変換

In [159]:
res = sql.toArray()

In [169]:
res

[(0, 'datetime', 'varchar(255)', 0, None, 0),
 (1, 'dateval', 'int', 0, None, 0),
 (2, 'ask', 'real', 0, None, 0),
 (3, 'bid', 'real', 0, None, 0)]

In [170]:
res = sql.toDataFrame()

In [172]:
res["dateval"].as_matrix()

array([  2.01704010e+13,   2.01704010e+13,   2.01704010e+13, ...,
         2.01704291e+13,   2.01704291e+13,   2.01704291e+13])

In [164]:
sql.toCSV("../data/res.csv")

In [173]:
list(res.index)

['2017-04-01 00:52:13',
 '2017-04-01 00:52:18',
 '2017-04-01 00:52:24',
 '2017-04-01 00:52:29',
 '2017-04-01 00:52:34',
 '2017-04-01 00:52:39',
 '2017-04-01 00:52:44',
 '2017-04-01 00:52:49',
 '2017-04-01 00:52:54',
 '2017-04-01 00:52:59',
 '2017-04-01 00:53:05',
 '2017-04-01 00:53:10',
 '2017-04-01 00:53:15',
 '2017-04-01 00:53:20',
 '2017-04-01 00:53:25',
 '2017-04-01 00:53:30',
 '2017-04-01 00:53:35',
 '2017-04-01 00:53:40',
 '2017-04-01 00:53:46',
 '2017-04-01 00:53:51',
 '2017-04-01 00:53:56',
 '2017-04-01 00:54:01',
 '2017-04-01 00:54:06',
 '2017-04-01 00:54:11',
 '2017-04-01 00:54:16',
 '2017-04-01 00:54:21',
 '2017-04-01 00:54:27',
 '2017-04-01 00:54:32',
 '2017-04-01 00:54:37',
 '2017-04-01 00:54:42',
 '2017-04-01 00:54:47',
 '2017-04-01 00:54:52',
 '2017-04-01 00:54:57',
 '2017-04-01 00:55:02',
 '2017-04-01 00:55:07',
 '2017-04-01 00:55:13',
 '2017-04-01 00:55:18',
 '2017-04-01 00:55:23',
 '2017-04-01 00:55:28',
 '2017-04-01 00:55:33',
 '2017-04-01 00:55:38',
 '2017-04-01 00: